# Fitting the synthetic STF to evaluate the seismic moment and source duration

We fit the synthetic STF to the preprocessed displacement pulse to evaluate the source parameters.

2024.11.19 Kurama Okubo

- 2024.11.25 update denoise method
- 2025.2.6 update the Tw_init with iteration to avoid the local minimum.

In [1]:
import os
import obspy
from obspy import read, Stream, Trace
from scipy import signal
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
%matplotlib inline
import glob
from glob import glob
import numpy as np
import mpmath as mp
import pandas as pd
import datetime
from datetime import timedelta
from tqdm import tqdm
import warnings

from matplotlib import gridspec

from scipy import interpolate
from scipy.optimize import curve_fit  
import matplotlib as mpl
import pickle
import copy

import seaborn as sns 
from scipy.interpolate import LSQUnivariateSpline
from scipy import integrate
from scipy.interpolate import CubicSpline

import h5py # store the STF in hdf5
from scipy.optimize import minimize

from STFfit_func import *

plt.rcParams["font.family"] = 'Arial'
# plt.rcParams["font.sans-serif"] = "DejaVu Sans, Arial, Helvetica, Lucida Grande, Verdana, Geneva, Lucid, Avant Garde, sans-serif"
plt.rcParams["font.size"] = 12
plt.rcParams["xtick.direction"] = "in"
plt.rcParams["xtick.major.size"] = 4.75
plt.rcParams["xtick.major.width"] = 0.75
plt.rcParams["xtick.minor.size"] = 3
plt.rcParams["xtick.minor.width"] = 0.4
plt.rcParams["xtick.minor.visible"] = True

plt.rcParams["ytick.direction"] = "in"
plt.rcParams["ytick.major.size"] = 4.75
plt.rcParams["ytick.major.width"] = 0.75
plt.rcParams["ytick.minor.size"] = 3
plt.rcParams["ytick.minor.width"] = 0.4
plt.rcParams["ytick.minor.visible"] = True

plt.rcParams["savefig.transparent"] = True
plt.rcParams['axes.linewidth'] = 0.75

from obspy.core.utcdatetime import UTCDateTime  
os.environ['TZ'] = 'GMT' # change time zone to avoid confusion in unix_tvec conversion
UTCDateTime.DEFAULT_PRECISION = 8

In [2]:
dataoutdir = "../data/04_STFfit"
if not os.path.exists(dataoutdir):
    os.makedirs(dataoutdir)

In [3]:
figdir = "../figure/debug_04_STFfit/"
if not os.path.exists(figdir):
    os.makedirs(figdir)

# Read the STF

In [4]:
gougepatch_id = "G3" # to set output filename
denoise_method = "detrend"
Qinv_quart = 50
k_waterlevel = 0.3
fo = h5py.File(f"../data/03_computePdisp/STF_all_{gougepatch_id}_Q{Qinv_quart}_wlv_{k_waterlevel:.2f}_denoisemethod_{denoise_method.lower()}.hdf5", 'r+')


In [5]:
# selected AE sensors for patch G3
AEsensor_list = ["OL23", "OL07", "OL08", "OL22"] # update: we use 4 close sensors "OL24"] 

In [6]:
tvec = np.array(fo[f"param/tvec_upsampled_margintrimmed"])

# pwin_pre = fo[f"param"].attrs["pwin_pre"]
dt_upsampled = fo[f"param"].attrs["dt_upsampled"]
pwin_pre = fo[f"param"].attrs["pwin_pre"]
pwin_len = fo[f"param"].attrs["pwin_len"] 


In [7]:
datacases = list(fo.keys())
datacases.remove("param")
# datacases

## Read source parameters from full waveform fitting

In [8]:
# Path to the best fit source parameters
bestfitsourceparam_finame = f"../../SourceInvFit/data/datacsv/gridsearch_bestparam_M0andTR_fb03-087.csv"

df_bestparam = pd.read_csv(bestfitsourceparam_finame, index_col=0) # from waveform inversion
datacases = df_bestparam.index
df_bestparam.head()

,VR_P_best,M0_best,TR_best,rake_best
fb03-087__0004,0.89,0.57,3.3,0
fb03-087__0009,0.73,0.05,3.1,0
fb03-087__0018,0.87,0.61,3.5,0
fb03-087__0019,0.79,0.05,3.1,0
fb03-087__0020,0.89,0.77,3.3,0


In [9]:
# # load color dictionary consistent to the plot of the repeated waveforms
# repeated_sensor_lcdict = "OL08" # the color dict is same for all the sensor although separately saved.
# with open(f'../../ComputeScaling/data/01_plot_gougeevents/lc_dict_{gougepatch_id}_{repeated_sensor_lcdict}.pkl', 'rb') as fi:
#     lc_dict = pickle.load(fi)

In [10]:
# Color for the all the events
skipcolor = 10
c_norm = mpl.colors.Normalize(vmin=0, vmax=len(datacases)+skipcolor)
lc_all = dict()
cmap = sns.color_palette("viridis_r", as_cmap=True)

for i, datacase in enumerate(datacases[::-1]):
    lc_all[datacase] = cmap(c_norm(i+skipcolor))


# Fit the STF to the displacement pulse

The initial parameters are crucial for the convergence of the optimization. We use the half-maximum amplitude width evaluated by the LBA and the peak of STF as the initial $T_R$.

## 1. Without deconvolution of attenuation

In [11]:
# fitting parameters
# TR_init = 2.8e-6 # Initial condition: This needs to close to the bestfit solution   
Tshift_init = 0.0
bounds = [(0, 10), (1.0e-6, 10.0e-6), (-0.5e-6, 1e-6)]
stf_type = "cosine" #"kupper" #"cosine"
xatol = 1e-2 # 1e-8 
fatol = 1e-2 # 1e-8 

residu_win = [0.5, 0.25]

# M0_init_factor = 0.8 # accounting for the effect of attenuation factor deconvolution
# TR_init_factor = 0.8 # accounting for the effect of attenuation factor deconvolution

TR_peakwinlen = 2.5e-6 #

for stnm in AEsensor_list:
    
    figdir_sensor = f"../figure/04_STFfit/noQcorr/{stnm}/{stf_type}/"
    
    if not os.path.exists(figdir_sensor):
        os.makedirs(figdir_sensor)

    for datacase in tqdm(datacases):
        gougeevent_id = int(datacase.split("__")[1])
        STF_sensor = np.array(fo[f"{datacase}/{stnm}/noQcorr/M0dot"])
        LBA_amp = 0.0 # this is zero as STF is normalized by LBA
        
        # LBA_amp = fo[f"{datacase}/{stnm}/noQcorr"].attrs["LBA_amp"] # this is zero as STF is normalized by LBA

        # M0_wavfit = df_bestparam.loc[datacase, :]["M0_best"]

        # TR_wavfit = df_bestparam.loc[datacase, :]["TR_best"]*1e-6*TR_init_factor
        # # Set the TR init as the double of peak distance
        # st_pmax = int(pwin_pre/dt_upsampled)
        # et_pmax = st_pmax + int(TR_peakwinlen/dt_upsampled)
        # pmax_ind = np.argmax(STF_sensor[st_pmax:et_pmax])
        # TR_init = 2*(pmax_ind*dt_upsampled)
        
        # Set TR as the width of the half-maximum level between LBA and peak
        st_pmax = int(pwin_pre/dt_upsampled)
        et_pmax = st_pmax + int(TR_peakwinlen/dt_upsampled)
        pmax = np.max(STF_sensor[st_pmax:et_pmax])
        pmax_ind = st_pmax + np.argmax(STF_sensor[st_pmax:et_pmax])
        halfamp = (LBA_amp + pmax)/2
        
        # search the half pulse width
        halfamp_list = np.where(np.diff(np.sign(STF_sensor - halfamp)))[0]
        
        # for tiny events, skip if we cannot find the LHA or RHA due to low S/N
        try:
            LHA_ind = halfamp_list[np.where(halfamp_list - pmax_ind < 0)[0][-1]]
            RHA_ind = halfamp_list[np.where(halfamp_list - pmax_ind > 0)[0][0]] # set HMPW just below the half-maximum amplitude
            HMPW = tvec[RHA_ind] - tvec[LHA_ind]
            TR_init = 2*HMPW
        except:
            TR_init = 2.0e-6 # case for HMPW not found
            # print(f"{stnm} {datacase} {LHA_ind} {RHA_ind} TR is set as {TR_init*1e6:.1f}μs")
            print(f"{stnm} {datacase} TR is set as {TR_init*1e6:.1f}μs")

        # Set the M0 fit by 0.5*(TR_init*peak_amp)
        M0_init = 0.5*pmax*TR_init
        # print(M0_init)
        # print(TR_init)

        # optimize the model parameters
        #https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html#scipy.optimize.minimize
 
        # Update 2025.2.6: to avoid the jump in the residual during the grid search caused by the dependency of the initial value,
        # we iterate to find the global minimum in the fitting.
        TR_init_list = TR_init * np.linspace(1.0, 1.1, 3)
        res_fun_all = []
    
        for TR_init_test in TR_init_list:
            x0 = [M0_init, TR_init_test, Tshift_init]
            # print(x0)
            
            # fit the synthetic STF to the dynamic rupture STF
            res_test = minimize(compute_res, x0, args=(STF_sensor, dt_upsampled, pwin_pre, residu_win, stf_type, False), 
                           method='Nelder-Mead', bounds=bounds, options={"return_all": False, "xatol":xatol, "fatol":fatol, "maxfev":1000})
            res_fun_all.append(res_test.fun)
        
        # use Tw_init with minimum residual 
        TR_init_best = TR_init_list[np.argmin(res_fun_all)]
        x0 = [M0_init, TR_init_best, Tshift_init]
        res = minimize(compute_res, x0, args=(STF_sensor, dt_upsampled, pwin_pre, residu_win, stf_type, False), 
                    method='Nelder-Mead', bounds=bounds, options={"return_all": False, "xatol":xatol, "fatol":fatol, "maxfev":1000})
    
        # x0 = [M0_init, TR_init, Tshift_init]
        

        # res = minimize(compute_res, x0, args=(STF_sensor, dt_upsampled, pwin_pre, residu_win, stf_type, False), 
        #                method='Nelder-Mead', bounds=bounds, options={"return_all": False, "xatol":xatol, "fatol":fatol, "maxfev":1000})

        # debug plot
        fig, ax = plt.subplots(1, 1, figsize=(6, 5))
        datacase
        ax.plot(tvec*1e6, STF_sensor/1e6, c=lc_all[datacase])
        
        # best-fit synthetic STF
        M0_best, TR_best, tshift_best = res.x
        tvec_syn = np.linspace(0, TR_best, int(TR_best/dt_upsampled))
        
        if stf_type=="kupper":
            STF_bestfit = stf_kupper(tvec_syn, TR_best, M0_best)
        elif stf_type=="cosine":
            STF_bestfit = stf_cosine(tvec_syn, TR_best, M0_best)
        else:
            raise ValueError("stf_type not found")

        max_STF_bestfit = STF_bestfit.max()
        error_fraction = res.fun/max_STF_bestfit

        SNR_sensor = fo[f"{datacase}/{stnm}/noQcorr"].attrs["SNR_sensor"] 
        ax.text(0.05, 0.6, f"event ID:{gougeevent_id}\n{res.success}\n$M_0$={M0_best:.2f}Nm\n$T_R$={TR_best*1e6:.2f}μs\n$Tshift$={tshift_best*1e6:.2f}μs\nRMSE={res.fun:.1f}Nm/s\nerror={error_fraction*100:.2f}%\nSNR={SNR_sensor:.1f}", transform=ax.transAxes)
        
        
        ax.plot((tvec_syn + pwin_pre + tshift_best)*1e6, STF_bestfit/1e6, c="r")
        ax.set_xlabel("Time [μs]")

        ylabelstr = r"$\mathrm{\dot{M}_0}$"
        ax.set_ylabel(f"{ylabelstr} [MNm/s]")
        ax.set_title("no Q correction")
        fig.tight_layout()

        plt.savefig(figdir_sensor + f"/debug_STFfit_{gougepatch_id}_{stnm}_{stf_type}_event{gougeevent_id}.png", dpi=100)

        plt.clf()
        plt.close()

        # store the data
        if not stf_type in fo[f"{datacase}/{stnm}/noQcorr"].keys():
            fo[f"{datacase}/{stnm}/noQcorr"].create_group(stf_type)
            
        fo[f"{datacase}/{stnm}/noQcorr/{stf_type}"].attrs["M0_best"] = M0_best
        fo[f"{datacase}/{stnm}/noQcorr/{stf_type}"].attrs["TR_best"] = TR_best
        fo[f"{datacase}/{stnm}/noQcorr/{stf_type}"].attrs["tshift_best"] = tshift_best
        fo[f"{datacase}/{stnm}/noQcorr/{stf_type}"].attrs["stf_type"] = stf_type
        fo[f"{datacase}/{stnm}/noQcorr/{stf_type}"].attrs["max_STF_bestfit"] = max_STF_bestfit
        fo[f"{datacase}/{stnm}/noQcorr/{stf_type}"].attrs["fitting_flag"] = res.success
        fo[f"{datacase}/{stnm}/noQcorr/{stf_type}"].attrs["RMSE"] = res.fun
        fo[f"{datacase}/{stnm}/noQcorr/{stf_type}"].attrs["error_fraction"] = error_fraction


100%|███████████████████████████████████████████| 44/44 [01:01<00:00,  1.41s/it]
/var/folders/6n/nsqc1sbn1kn5ccc7y51plgz00000gn/T/ipykernel_69845/1433040553.py:78: OptimizeWarning: Initial guess is not within the specified bounds
  res_test = minimize(compute_res, x0, args=(STF_sensor, dt_upsampled, pwin_pre, residu_win, stf_type, False),
/var/folders/6n/nsqc1sbn1kn5ccc7y51plgz00000gn/T/ipykernel_69845/1433040553.py:85: OptimizeWarning: Initial guess is not within the specified bounds
  res = minimize(compute_res, x0, args=(STF_sensor, dt_upsampled, pwin_pre, residu_win, stf_type, False),
100%|███████████████████████████████████████████| 44/44 [01:10<00:00,  1.60s/it]


## 2. With deconvolution of the attenuation factor

In [12]:
datacases[37]

'fb03-087__0111'

In [13]:
fo[f"param"].keys()

<KeysViewHDF5 ['tvec_upsampled_margintrimmed']>

In [14]:
# fitting parameters
# TR_init = 2.8e-6 # Initial condition: This needs to close to the bestfit solution   
Tshift_init = 0.0
bounds = [(0, 10), (1.0e-6, 10.0e-6), (-0.5e-6, 1e-6)]
stf_type = "cosine" #"kupper" #"cosine"
xatol = 1e-2 #1e-8 #2 
fatol = 1e-2 #1e-8 #2

residu_win = [0.5, 0.25]

# M0_init_factor = 0.8 # accounting for the effect of attenuation factor deconvolution
# TR_init_factor = 0.8 # accounting for the effect of attenuation factor deconvolution

TR_peakwinlen = 2.5e-6 #

k_waterlevel = fo[f"param"].attrs["k_waterlevel"] 

for stnm in AEsensor_list:
    
    figdir_sensor = f"../figure/04_STFfit/Q{Qinv_quart}_{k_waterlevel:.2f}/{stnm}/{stf_type}/"
    
    if not os.path.exists(figdir_sensor):
        os.makedirs(figdir_sensor)

    for datacase in tqdm(datacases):
        gougeevent_id = int(datacase.split("__")[1])
        STF_sensor = np.array(fo[f"{datacase}/{stnm}/Q{Qinv_quart}/M0dot"])
        LBA_amp = fo[f"{datacase}/{stnm}/Q{Qinv_quart}"].attrs["LBA_amp"] # this is zero as STF is normalized by LBA

        # M0_wavfit = df_bestparam.loc[datacase, :]["M0_best"]

        # TR_wavfit = df_bestparam.loc[datacase, :]["TR_best"]*1e-6*TR_init_factor
        # # Set the TR init as the double of peak distance
        # st_pmax = int(pwin_pre/dt_upsampled)
        # et_pmax = st_pmax + int(TR_peakwinlen/dt_upsampled)
        # pmax_ind = np.argmax(STF_sensor[st_pmax:et_pmax])
        # TR_init = 2*(pmax_ind*dt_upsampled)
        
        # Set TR as the width of the half-maximum level between LBA and peak
        st_pmax = int(pwin_pre/dt_upsampled)
        et_pmax = st_pmax + int(TR_peakwinlen/dt_upsampled)
        pmax = np.max(STF_sensor[st_pmax:et_pmax])
        pmax_ind = st_pmax + np.argmax(STF_sensor[st_pmax:et_pmax])
        halfamp = (LBA_amp + pmax)/2
        
        # search the half pulse width
        halfamp_list = np.where(np.diff(np.sign(STF_sensor - halfamp)))[0]
        
        # for tiny events, skip if we cannot find the LHA or RHA due to low S/N
        try:
            LHA_ind = halfamp_list[np.where(halfamp_list - pmax_ind < 0)[0][-1]]
            RHA_ind = halfamp_list[np.where(halfamp_list - pmax_ind > 0)[0][0]] # set HMPW just below the half-maximum amplitude
            HMPW = tvec[RHA_ind] - tvec[LHA_ind]
            TR_init = 2*HMPW
        except:
            TR_init = 2.0e-6 # case for HMPW not found
            print(f"{stnm} {datacase} {LHA_ind} {RHA_ind} TR is set as {TR_init*1e6:.1f}μs")

        # Set the M0 fit by 0.5*(TR_init*peak_amp)
        M0_init = 0.5*pmax*TR_init
        # print(M0_init)
        # print(TR_init)
        x0 = [M0_init, TR_init, Tshift_init]
        
        # optimize the model parameters
        #https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html#scipy.optimize.minimize
 
        res = minimize(compute_res, x0, args=(STF_sensor, dt_upsampled, pwin_pre, residu_win, stf_type, False), 
                       method='Nelder-Mead', bounds=bounds, options={"return_all": False, "xatol":xatol, "fatol":fatol, "maxfev":1000})

        # debug plot
        fig, ax = plt.subplots(1, 1, figsize=(6, 5))
        datacase
        ax.plot(tvec*1e6, STF_sensor/1e6, c=lc_all[datacase])
        
        # best-fit synthetic STF
        M0_best, TR_best, tshift_best = res.x
        tvec_syn = np.linspace(0, TR_best, int(TR_best/dt_upsampled))
        
        if stf_type=="kupper":
            STF_bestfit = stf_kupper(tvec_syn, TR_best, M0_best)
        elif stf_type=="cosine":
            STF_bestfit = stf_cosine(tvec_syn, TR_best, M0_best)
        else:
            raise ValueError("stf_type not found")

        max_STF_bestfit = STF_bestfit.max()
        error_fraction = res.fun/max_STF_bestfit

        SNR_sensor = fo[f"{datacase}/{stnm}/Q{Qinv_quart}"].attrs["SNR_sensor"] 
        ax.text(0.05, 0.6, f"event ID:{gougeevent_id}\n{res.success}\n$M_0$={M0_best:.2f}Nm\n$T_R$={TR_best*1e6:.2f}μs\n$Tshift$={tshift_best*1e6:.2f}μs\nRMSE={res.fun:.1f}Nm/s\nerror={error_fraction*100:.2f}%\nSNR={SNR_sensor:.1f}", transform=ax.transAxes)
        
        
        ax.plot((tvec_syn + pwin_pre + tshift_best)*1e6, STF_bestfit/1e6, c="r")
        ax.set_xlabel("Time [μs]")

        ylabelstr = r"$\mathrm{\dot{M}_0}$"
        ax.set_ylabel(f"{ylabelstr} [MNm/s]")
        ax.set_title(f"Q{Qinv_quart}")

        ax.set_xlim([10, 35])
        
        fig.tight_layout()

        plt.savefig(figdir_sensor + f"/debug_STFfit_{gougepatch_id}_{stnm}_{stf_type}_event{gougeevent_id}.png", dpi=100)

        plt.clf()
        plt.close()

        # store the data
        if not stf_type in fo[f"{datacase}/{stnm}/Q{Qinv_quart}"].keys():
            fo[f"{datacase}/{stnm}/Q{Qinv_quart}"].create_group(stf_type)
            
        fo[f"{datacase}/{stnm}/Q{Qinv_quart}/{stf_type}"].attrs["M0_best"] = M0_best
        fo[f"{datacase}/{stnm}/Q{Qinv_quart}/{stf_type}"].attrs["TR_best"] = TR_best
        fo[f"{datacase}/{stnm}/Q{Qinv_quart}/{stf_type}"].attrs["tshift_best"] = tshift_best
        fo[f"{datacase}/{stnm}/Q{Qinv_quart}/{stf_type}"].attrs["stf_type"] = stf_type
        fo[f"{datacase}/{stnm}/Q{Qinv_quart}/{stf_type}"].attrs["max_STF_bestfit"] = max_STF_bestfit
        fo[f"{datacase}/{stnm}/Q{Qinv_quart}/{stf_type}"].attrs["fitting_flag"] = res.success
        fo[f"{datacase}/{stnm}/Q{Qinv_quart}/{stf_type}"].attrs["RMSE"] = res.fun
        fo[f"{datacase}/{stnm}/Q{Qinv_quart}/{stf_type}"].attrs["error_fraction"] = error_fraction


100%|███████████████████████████████████████████| 44/44 [00:16<00:00,  2.67it/s]


In [15]:
# res

In [16]:
res.fun

6045.5077588924605

In [17]:
fo.close()

# Conclusion

We computed the best-fit STF to the observed P wave pulse to evaluate the source parameters. We analyze the statistics of the source parameters in the next notebook.